In [2]:
from ephys_queries import select_spike_times
from ephys_queries import db_setup_core
from dotenv import load_dotenv
from pathlib import Path

In [3]:
from spiketimes.df.binning import binned_spiketrain_bins_provided
from spiketimes.df.statistics import auc_roc_test_by
from spiketimes.df.statistics import diffmeans_test_by
from spiketimes.df.baseline import zscore_standardise_by
from spiketimes.utils import p_adjust

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import auc

In [5]:
data_dir = Path(".").absolute().parent / "data"
fig_dir = Path(".").absolute().parent / "figs"

In [6]:
group_names = ["acute_citalopram", 
            "acute_saline", 
            "acute_cit", 
            "acute_sal"]
dfb = (
    pd.read_csv(data_dir / "baseline.csv")
    .loc[lambda x: x["group_name"].isin(group_names)]
)

In [7]:
block_name = "chal"
t_before = 10 * 60
fs = 30000
load_dotenv()
engine, metadata = db_setup_core()

df_spikes = (
    select_spike_times(
            engine, 
            metadata,
            group_names=group_names,
            block_name=block_name,
        t_before=t_before,
        align_to_block=True
            )
    .assign(spiketimes=lambda x: x["spike_time_samples"].divide(fs))
)

In [8]:
df_spikes.head(3)

,neuron_id,spike_time_samples,group_name,spiketimes
0,1011,-17996636,acute_cit,-599.887867
1,1011,-17992217,acute_cit,-599.740567
2,1011,-17987075,acute_cit,-599.569167


### Define Responders

In [31]:
def response_dm(row, p_col="p_dm", diff_mean_col="diff_of_means"):
    if row[p_col] >= 0.05:
        return "Non Responder"
    elif row[diff_mean_col] < 0:
        return "Inhibited"
    else:
        return "Stimulated"
    
    
def response_auc(row):
    if row["p_dm"] >= 0.05:
        return "Non Responder"
    elif row["AUCROC"] < 0.5:
        return "Inhibited"
    else:
        return "Stimulated"
    
def percent_baseline(df):
    mean_baseline = df.loc[lambda x: x.time < 0].spike_counts.mean()
    return df.assign(pct_baseline=lambda x: x.spike_counts.divide(mean_baseline) * 100)

In [63]:
binsize = 30
bins = np.arange(-600, 1260, binsize)

df = (
    binned_spiketrain_bins_provided(
        df_spikes, 
        bins, 
        spiketimes_col="spiketimes",
                               
        spiketrain_col="neuron_id"
    )
    .rename(columns={"counts": "spike_counts", "bin": "time"})
    .merge(dfb[["neuron_id", "cluster", "group_name"]])
    .assign(treatment=lambda x: x["group_name"].map({
                        "acute_cit": "Citalopram",
                        "acute_citalopram": "Citalopram",
                        "acute_saline": "Saline",
                        "acute_sal": "Saline"}),
           condition=lambda x: x["time"].apply(lambda y: "Baseline" if y < 0 else "Challenge"),
           firing_rate=lambda x: x["spike_counts"].divide(60),
           cond=lambda x: x["condition"].map({"Baseline":0, "Challenge":1}),

        )
    .drop("group_name", axis=1)
    .groupby("neuron_id")
    .apply(lambda x: percent_baseline(x))
    .reset_index(drop=True)
)


df.head()

,neuron_id,time,spike_counts,cluster,treatment,condition,firing_rate,cond,pct_baseline
0,1011,-600,222,slow_irregular,Citalopram,Baseline,3.700000,0,79.712747
1,1011,-570,220,slow_irregular,Citalopram,Baseline,3.666667,0,78.994614
2,1011,-540,272,slow_irregular,Citalopram,Baseline,4.533333,0,97.666068
3,1011,-510,287,slow_irregular,Citalopram,Baseline,4.783333,0,103.052065
4,1011,-480,323,slow_irregular,Citalopram,Baseline,5.383333,0,115.978456


In [64]:
min_fr = 0.5

df = (
    df
    .groupby("neuron_id")
    .filter(lambda x: x.loc[x.time < 0].spike_counts.divide(binsize).mean() > min_fr)
)

In [65]:
res_auc = auc_roc_test_by(df, n_boot=100, 
                      spiketrain_col="neuron_id", 
                      spikecount_col="firing_rate",
                     condition_col="cond")

In [66]:
res_dm = diffmeans_test_by(df, n_boot=1000, 
                      spiketrain_col="neuron_id", 
                      spikecount_col="firing_rate",
                     condition_col="cond")

In [67]:
res_pct = diffmeans_test_by(df, n_boot=100, 
                      spiketrain_col="neuron_id", 
                      spikecount_col="pct_baseline",
                     condition_col="cond").rename(columns={"diff_of_means": "diff_of_means_pct"})

In [68]:
res_auc["p_AUCROC"] = p_adjust(res_auc["p"])
res_dm["p_dm"] = p_adjust(res_dm["p"])
res_pct["p_pct"] = p_adjust(res_dm["p"])

df_effects = (
    res_auc
    .merge(df[["neuron_id", "cluster", "treatment"]]
              .drop_duplicates())
    .drop("p", axis=1)
    .merge(res_dm)
    .drop("p", axis=1)
    .merge(res_pct)
    .drop("p", axis=1)
    .assign(
        response_dm=lambda x: x.apply(response_dm, axis=1),
        response_pct=lambda x: x.apply(response_dm, p_col="p_pct", diff_mean_col="diff_of_means_pct", axis=1),
        response_aucroc=lambda x: x.apply(response_auc, axis=1)
    )
)

### Get Z Score Time Series

In [74]:
bins = np.arange(-600, 1200, 30)
df_zscores = (
    binned_spiketrain_bins_provided(df_spikes, bins=bins, spiketimes_col="spiketimes", spiketrain_col="neuron_id")
    .merge(df_effects[["response_dm", "cluster", "treatment", "neuron_id"]], how="right")
    .pipe(lambda x: zscore_standardise_by(df=x, baseline_start_stop=np.array([-600, 0]),
                     spiketrain_col="neuron_id", data_col="counts", time_col="bin"))

)

In [75]:
def percent_baseline(df):
    mean_baseline = df.loc[lambda x: x.bin < 0].counts.mean()
    return df.assign(pct_base = lambda x: x.counts.divide(mean_baseline) * 100)

bins = np.arange(-600, 1200, 60)

df_pct_base = (
    binned_spiketrain_bins_provided(df_spikes, bins=bins, spiketimes_col="spiketimes", spiketrain_col="neuron_id")
    .groupby("neuron_id")
    .apply(percent_baseline)
    .reset_index(drop=True)
)

In [76]:
df_ts = pd.merge(df_zscores, df_pct_base)

In [78]:
df_effects = (
    df_zscores.groupby("neuron_id")
    .apply(lambda x: auc(np.arange(len(x)), x["zscore"]))
    .reset_index()
    .rename(columns={0:"auc"})
    .merge(df_effects[["neuron_id", "cluster", "treatment", "response_dm", 
                       "response_aucroc", "diff_of_means", "response_pct",
                       "diff_of_means_pct"]])
)

In [80]:
df_effects.to_csv(data_dir / "cit_effects.csv", index=False)
df_ts.to_csv(data_dir / "cit_timeseries.csv", index=False)

In [79]:
df_effects

,neuron_id,auc,cluster,treatment,response_dm,response_aucroc,diff_of_means,response_pct,diff_of_means_pct
0,1011,-18.674011,slow_irregular,Citalopram,Non Responder,Non Responder,-0.691260,Non Responder,-14.892499
1,1013,-228.472329,slow_regular,Citalopram,Inhibited,Inhibited,-0.879898,Inhibited,-49.087775
2,1014,-6.920328,slow_regular,Citalopram,Non Responder,Non Responder,-0.177988,Non Responder,-41.312450
3,1015,-50.383443,slow_regular,Citalopram,Inhibited,Inhibited,-0.993008,Inhibited,-48.126404
4,1019,-80.096444,slow_irregular,Citalopram,Inhibited,Inhibited,-0.460894,Inhibited,-29.799201
...,...,...,...,...,...,...,...,...,...
291,2627,27.396513,slow_irregular,Saline,Non Responder,Non Responder,0.075610,Non Responder,16.802168
292,2628,-6.490980,no_baseline,Saline,Non Responder,Non Responder,-0.008638,Non Responder,-2.692430
293,2629,-94.403555,slow_irregular,Saline,Inhibited,Inhibited,-1.016667,Inhibited,-64.893617
294,2630,26.439028,fast_firing,Saline,Non Responder,Non Responder,0.294715,Non Responder,4.344699
